In [ ]:
%pip install langkit[all]==0.0.24.dev0

Let's say we want the standard LLM metrics for English prompts, but we don't want any metrics on the responses since they're in a different unsupported language. We can use the default configurations for the prompt metrics and set the response metric configurations to `None` to turn them off.


In [2]:
import langkit.llm_metrics as llmm
from langkit import LangKitConfig

no_response_config = LangKitConfig(
    prompt_languages={"", "en"},
    response_languages=set(),
    response_pattern_file_path=None,  # regexes are semi-international, could leave this as default
    response_sentiment_lexicon=None,
    response_theme_file_path=None,
    response_topic_model_path=None,
    response_toxicity_model_path=None,
    response_transformer_name=None,
)
schema = llmm.init(config=no_response_config)

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [3]:
import whylogs as why
from langkit.whylogs.samples import load_chats, show_first_chat

chats = load_chats()
results = why.log(chats, schema=schema)

for column in results.view().get_columns().keys():
  print(column)

⚠️ No session found. Call whylogs.init() to initialize a session and authenticate. See https://docs.whylabs.ai/docs/whylabs-whylogs-init for more information.
prompt
response
prompt.has_patterns
prompt.sentiment_nltk
prompt.flesch_reading_ease
prompt.automated_readability_index
prompt.aggregate_reading_level
prompt.syllable_count
prompt.lexicon_count
prompt.sentence_count
prompt.character_count
prompt.letter_count
prompt.polysyllable_count
prompt.monosyllable_count
prompt.difficult_words
prompt.jailbreak_similarity
prompt.toxicity
